In [4]:
#import statements
import csv, random
from copy import deepcopy

In [16]:
#termination criteria is when fitness < 0.01
COUNT = [10]
        
class Tree:
    #initialize
    print("initialize tree")
    # make sure tree class is immutable
    def __init__(self):
        #initial tree -> call random tree generator
        #mutated tree -> call clone; makes copy of tree with changes
        # Node root
        self.root = Node()
        self.root.value = self.randomOp()
        self.array = []
        #self.depth = 0
        self.operators = ['+', '-', '*', '/']
        self.operands = ['x', '-2', '-1', '1', '2']
        self.fitness = 0
        self.weights = self.makeWeights()

        self._initialTree(self.root)
        
    def makeWeights(self):
        weightArr = []
        for i in range(100):
            if i <= 20:
                weightArr.append(20)
            elif i > 20 & i<=40:
                weightArr.append(10)
            elif i > 40 & i<=60:
                weightArr.append(5)
            elif i > 60 & i<=80:
                weightArr.append(2)
            elif i > 80 & i<=100:
                weightArr.append(1)
        return weightArr
        
    def randomOp(self):
        options = ['+', '-', '*', '/', 'x', '-2', '-1', '1', '2']
        length = len(options)
        #generate random operation
        value = random.randint(0,length-1)
        return options[value]
    
    def firstVal(self):
        start = Node()
        #set root of tree to first val
        self.root = start

        #generate random operation
        val = self.randomOp()
        #set node value
        start.value = val
        print(start.value)
        return start

    def _initialTree(self, node):
        operators = ['+', '-', '*', '/']
        operands = ['x', '-2', '-1', '1', '2']
        
        if (node.value in operators):
            #print("enter operator = ", node.value)
            
            leftNode = Node()
            leftNode.value = self.randomOp()
            node.left = leftNode
            leftNode.parent = node
            
            rightNode = Node()
            rightNode.value = self.randomOp()
            node.right = rightNode
            rightNode.parent = node
            
            self._initialTree(leftNode)
            self._initialTree(rightNode)

    def initialPop(self):
        #calls initial tree to create initial population
        forrest = []
        for i in range(10):
            newTree = Tree()
            forrest.append(newTree)
        #print(forrest[0].root.value)

    def printIO(self, node):
        #prints in order traversal of the tree
        if node:
            self.printIO(node.left)

            print(node.value)

            self.printIO(node.right)
    
    def treeToArray(self, root):
        #adds elements of tree to array
        if root is None:
            return
        if root:
            self.treeToArray(root.left)

            self.array.append(root)

            self.treeToArray(root.right)
    
    def randomNode(self):
        #picks random node from the tree
        length = len(self.array)
        #print("tree array: ", tree.array)
        #print("length: ", length)
        if length == 1:
            return self.array[0]
        value = random.randint(0,length-1)
        return self.array[value]

    def IOtraversal():
        # IN ORDER TREE TRAVERSAL
        # looks at node the second time you hit it
        # build string as you traverse; when you hit leaf node, add to string
        # when you hit parent for second time, add to string
        return
    
    def crossover(self, tree2):
        #takes two trees and swaps parent tree 1 with parent tree 2

        #makes copy of tree to crossover
        dup1 = deepcopy(self)
        dup1.treeToArray(dup1.root)
        dup2 = deepcopy(tree2)
        dup2.treeToArray(dup2.root)

        #find random node from tree
        random1 = dup1.randomNode()
        #print("random1 node: ", random1.value)
        random2 = dup2.randomNode()
        #print("random2 node: ", random2.value)

        #check parents of random nodes and change pointers to right or left
        if random1.parent == None:
            dup1.root = random2
        elif random1.parent.left == random1:
            random1.parent.left = random2
        else:
            random1.parent.right = random2
        
        if random2.parent == None:
            dup2.root = random1
        elif random2.parent.left == random2:
            random2.parent.left = random1
        else:
            random2.parent.right = random1

        #switch pointers to parents between crossover nodes
        #print("original random 1 parent", random1.parent)
        #print("original random2 parent", random2.parent)

        tempParent = random1.parent
        random1.parent = random2.parent
        #print("Mod random 1 parent", random2.parent)
        random2.parent = tempParent
        #print("Mod random 2 parent", random2.parent)

        return dup1, dup2
    
    def mutate(self):
        operators = ['+', '-', '*', '/']
        operands = ['x', '-2', '-1', '1', '2']
        
        duplicate = deepcopy(self)
        duplicate.treeToArray(duplicate.root)
        
        random1 = duplicate.randomNode()
        
        #print("original = ", random1.value)
        
        if random1.value in operators:
            length = len(operators)
            value = random.randint(0,length-1)
            random1.value = operators[value]
        else:
            length = len(operands)
            value = random.randint(0,length-1)
            random1.value = operands[value]
            
        #print("mutated = ", random1.value)
        
        
        self._initialTree(random1)
        
        return duplicate
        
        #make sure an operator does not switch to an operand
    
    def print2DUtil(self, root, space):
        # Base case
        if (root == None):
            return
        # Increase distance between levels
        space += COUNT[0]
    
        # Process right child first
        self.print2DUtil(root.right, space)
    
        # Print current node after space
        # count
        print()
        for i in range(COUNT[0], space):
            print(end=" ")
        print(root.value)
    
        # Process left child
        self.print2DUtil(root.left, space)

    def print2D(self, root):
        # space=[0]
        # Pass initial space count as 0
        self.print2DUtil(root, 0) 
    
    def simplify(self, x, node):
        #operators = ['+', '-', '*', '/']
        #operands = ['x', '-2', '-1', '1', '2']
        val = 0

        if node:
            #print("node value", node.value)
            #check for x value
            if node.value == "x":
                node.value = x
            # check for operand
            if node.value not in self.operators:
                return float(node.value)
            # First recur on left child
            if node.left.value in self.operators:
                self.simplify(x, node.left)

            # the recur on right child
            if node.right.value in self.operators:
                self.simplify(x, node.right)
        
            # update x to given parameter value
            if node.left.value == "x":
                node.left.value = x
            if node.right.value == "x":
                node.right.value = x
            
            # applying operators
            if node.value == "+":
                node.value = float(node.left.value) + float(node.right.value)
            if node.value == "-":
                node.value = float(node.left.value) - float(node.right.value)
            if node.value == "*":
                node.value = float(node.left.value) * float(node.right.value)
            if node.value == "/":
                # WHAT TO DO WITH DIVISION AND ZEROS
                if float(node.right.value) == 0:
                    node.value = 1
                else:
                    node.value = float(node.left.value) / float(node.right.value)
            
            #break function and return value at root
            if node == self.root:
                return float(node.value)
            
    def evaluate(self, arr):
        #evaluates fitness of a tree
        #needs to be very efficient
        
        #returns float or int
        arr[0] = float(arr[0])
        arr[1] = float(arr[1])

        duplicate = deepcopy(self)
        duplicate.treeToArray(duplicate.root)

        val = duplicate.simplify(arr[0], duplicate.root)
        #print("return from simplify: ", val)
        output = (val - arr[1])**2
        
        return output
    
    def getFitness(self, data):
        total = 0
        #calculates fitness of each tree
        for i in range(len(data)):
            val = self.evaluate(data[i])
            total = total + val
            # divide by len of training
        self.fitness = total / len(data)
        

class Node:
    #initialize
    print("initialize node")
    def __init__(self):
        # Node parent, left, right
        self.parent = None
        self.left = None
        self.right = None
        # Value: contents of the node (type: str)
        self.value = None 
    

initialize tree
initialize node


In [19]:
def main():
    with open('dataset1.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile))
    data = data[1:]
    train = data[0:20000]
    test = data[20000:]
    miniTrain = data[0:10]
    
    print("TESTING EVAL")
    # build tree population of 100
    # read up on population sizes
    newGen = []
    for i in range(100):
        newGen.append(Tree())
    
    #producing 10 generations
    NUM_GENS = 0
    while NUM_GENS < 100:
        #print("NEWGEN", newGen[0:10])
        #loop through each tree
        for tree in range(len(newGen)):
            newGen[tree].getFitness(miniTrain)
            #define self.fitness in tree class
        
        #print("FITNESS VALS ARRAY", fitnessVals)
        #x.fitness
        newGen.sort(key=lambda x: x.fitness)
        print("BEST INDIV FROM NEW GEN", newGen[0].fitness)
        print()
        print()

        # crossover 80
        oldGen = newGen
        newGen = []
        for crossTree in range(40):
            # read on who to select for crossover
            children = []
            childWeights = random.choices(oldGen, weights = oldGen[0].weights, k = 2)

            #value = random.randint(0, len(oldGen)-1)
            
            # generates 10 trees from a crossover of 2 trees
            for brood in range(5):
                cross1, cross2 = childWeights[0].crossover(childWeights[1])
                children.append(cross1)
                children.append(cross2)
                
            # evaluates the fitness of the 10 children generated
            for child in children:
                child.getFitness(miniTrain)
            children.sort(key=lambda x: x.fitness)
            
            #newGen.append(crossed)
            newGen.append(children[0])
            newGen.append(children[1])
        #print("NEW GEN AFTER CROSSOVER", newGen[0:10])
        
        # mutate 10
        for mute in range(10):
            # randomize tree chosen for mutate -> fitter individuals have higher chance
            mutated = oldGen[mute].mutate()
            newGen.append(mutated)
        #print("NEW GEN AFTER MUTATE", newGen[0:10])

        # reproduce 10
        for repro in range(10):
            # picked with probability proportionate to fitness
            newGen.append(oldGen[repro])
        
        #print("NEW GEN ARRAY", newGen[0:10])
        print()
        print()
        NUM_GENS = NUM_GENS + 1
        #while loop end


if __name__ == "__main__":
    main()

TESTING EVAL
BEST INDIV FROM NEW GEN 162.1908737511532




BEST INDIV FROM NEW GEN 138.43254950999668




BEST INDIV FROM NEW GEN 62.1505044726044




BEST INDIV FROM NEW GEN 59.17485095928269




BEST INDIV FROM NEW GEN 21.311930000000007




BEST INDIV FROM NEW GEN 12.08793




KeyboardInterrupt: 